# FPP parametric scan

## Import packages

In [ ]:
using Revise
using FUSE
using Plots; gr();
FUSE.logging(Logging.Info; actors=Logging.Error);

### Setup distributed computing environment

See more details here: https://fuse.help/parallel.html

In [ ]:
FUSE.parallel_environment("localhost")
using Distributed
@everywhere using FUSE

### Define workflow and extent of the scan

In [ ]:
@everywhere function workflow(B0, spf)::IMAS.dd
    try
        ini, act = FUSE.case_parameters(:FPP; version=:v1_demount, init_from=:scalars);
        ini.equilibrium.ip *= B0/ini.equilibrium.B0
        ini.equilibrium.B0 = B0
        ini.core_profiles.polarized_fuel_fraction = spf
        dd = FUSE.init(ini, act);
        FUSE.ActorWholeFacility(dd, act);
        return dd
    catch e
        @warn e
        return IMAS.dd()
    end
end

ini, act = FUSE.case_parameters(:FPP; version=:v1_demount, init_from=:scalars);
nB0 = 3
nSPF = 3
cases = [[B0,spf] for B0 in ini.equilibrium.B0.*LinRange(0.5,2,nB0), spf in LinRange(0,1,nSPF)]

### Run simulations

In [ ]:
using ProgressMeter
DD = @showprogress pmap(case -> workflow(case...), cases);

### Extract quantities of interest from series of `dd`'s

In [ ]:
results=FUSE.extract(reshape(DD,length(DD)); filter_invalid=false)

### Plot scan results

In [ ]:
results_ = reshape(results.Pelectric_net,(nSPF,nB0))
B0_ = [B0 for B0 in ini.equilibrium.B0.*LinRange(0.5,2,nB0), spf in LinRange(0,1,nSPF)]
spf_ = [spf for B0 in ini.equilibrium.B0.*LinRange(0.5,2,nB0), spf in LinRange(0,1,nSPF)]
heatmap(spf_[1,:],B0_[:,1],results_,xlabel="SPF fraction",ylabel="B0",levels=60)